## Midterm 1 
### Professor Kutz
### EE 399
### By: Gerin George

In [101]:
import numpy as np
from scipy.io import loadmat
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

results=loadmat('yalefaces.mat')
X=results['X']

In [102]:
# Labeling faces from 0-9
lighting_scenes_per_face = 64
faces_to_label = 10

columns_per_face = lighting_scenes_per_face

labels = np.repeat(np.arange(faces_to_label), columns_per_face)

print(labels)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 

In [103]:
# Part A: 

lighting_scenes_per_face = 64
faces_to_extract = 10

columns_to_extract = faces_to_extract * lighting_scenes_per_face

first_10_faces = X[:, :columns_to_extract]

print(first_10_faces)

[[0.00392157 0.01740196 0.57573529 ... 0.18995098 0.52156863 0.        ]
 [0.00784314 0.00784314 0.53946078 ... 0.17107843 0.47205882 0.00661765]
 [0.00784314 0.00784314 0.48210784 ... 0.17696078 0.46593137 0.00784314]
 ...
 [0.51813725 0.0752451  0.03137255 ... 0.41372549 0.08823529 0.00539216]
 [0.51813725 0.07058824 0.03137255 ... 0.40147059 0.08112745 0.02205882]
 [0.50269608 0.05882353 0.03137255 ... 0.39215686 0.06911765 0.02745098]]


In [104]:
# Part B:

np.random.seed(42) 

lighting_scenes_per_face = 64
faces_to_extract = 10
test_scenes_per_face = 14
train_scenes_per_face = 50

# Shuffle lighting scenes within each face
for i in range(faces_to_extract):
    face_start = i * lighting_scenes_per_face
    face_end = (i + 1) * lighting_scenes_per_face
    np.random.shuffle(first_10_faces[:, face_start:face_end])

# Initialize training and test matrices
train_data = np.zeros((1024, faces_to_extract * train_scenes_per_face))
test_data = np.zeros((1024, faces_to_extract * test_scenes_per_face))

# Split the data into training and test sets
for i in range(faces_to_extract):
    face_start = i * lighting_scenes_per_face
    train_start = i * train_scenes_per_face
    test_start = i * test_scenes_per_face
    
    train_data[:, train_start:train_start + train_scenes_per_face] = first_10_faces[:, face_start:face_start + train_scenes_per_face]
    test_data[:, test_start:test_start + test_scenes_per_face] = first_10_faces[:, face_start + train_scenes_per_face:face_start + lighting_scenes_per_face]


In [105]:
# Part C:

# Perform PCA on the training data
n_modes = 20
pca = PCA(n_components=n_modes)
pca_train_data = pca.fit_transform(train_data.T)

# Create training labels
train_labels = np.repeat(np.arange(faces_to_extract), train_scenes_per_face)

# Perform LDA on the reduced training data
lda = LinearDiscriminantAnalysis()
lda.fit(pca_train_data, train_labels)

# Transform the test data using the same PCA model
pca_test_data = pca.transform(test_data.T)

# Create test labels
test_labels = np.repeat(np.arange(faces_to_extract), test_scenes_per_face)

# Make predictions on the reduced test data
predictions = lda.predict(pca_test_data)

# Calculate classification accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f"LDA Classification accuracy: {accuracy * 100:.2f}%")

LDA Classification accuracy: 66.43%


In [106]:
# Part D:

# Create training labels
train_labels = np.repeat(np.arange(faces_to_extract), train_scenes_per_face)

# Train SVM on the reduced training data
svm = SVC(random_state=42)
svm.fit(pca_train_data, train_labels)

# Transform the test data using the same PCA model
pca_test_data = pca.transform(test_data.T)

# Create test labels
test_labels = np.repeat(np.arange(faces_to_extract), test_scenes_per_face)

# Make predictions on the reduced test data
svm_predictions = svm.predict(pca_test_data)

# Calculate SVM classification accuracy
svm_accuracy = accuracy_score(test_labels, svm_predictions)
print(f"SVM classification accuracy: {svm_accuracy * 100:.2f}%")

# Train Decision Tree on the reduced training data
dt = DecisionTreeClassifier(random_state=42)
dt.fit(pca_train_data, train_labels)

# Make predictions on the reduced test data
dt_predictions = dt.predict(pca_test_data)

# Calculate Decision Tree classification accuracy
dt_accuracy = accuracy_score(test_labels, dt_predictions)
print(f"Decision Tree classification accuracy: {dt_accuracy * 100:.2f}%")

SVM classification accuracy: 84.29%
Decision Tree classification accuracy: 85.71%


LDA appears to be the worst at classifying faces with accuracy. SVM and Decision Tree perform similarly, although with different random states one can perform better than the other. For me, Decision Trees generally performed the best, although this margin is very small. LDA performs better when the data is linearly separable, which this is not. SVM and Decision Trees performed better because SVM is effective at handling hihg-dimensional data and can capture non-linear relationships better through the use of kernel functions. Decision trees can handle both linear and non-linear relationships, so it can perform better than LDA here. 